In [1]:
import pandas as pd
import sqlite3
import json 
import numpy as np

from pathlib import Path

In [2]:
file = Path("test_20240606.csv")
df = pd.read_csv(file)

In [3]:
list(df.columns)

['Unnamed: 0',
 'owner',
 'offtaker',
 'storage_energy',
 'storage_power',
 'url',
 'location',
 'developer',
 'offtake_desc',
 'epc',
 'gen_dc',
 'gen_ac',
 'cost',
 'cost_yr',
 'technology_general',
 'tech_module',
 'module_count',
 'tech_invert',
 'invert_count',
 'tracking',
 'storage',
 'storage_tech',
 'storage_config',
 'project_description',
 'name']

In [4]:
# Create a connection to the SQLite database
conn = sqlite3.connect('project_data.db')

# Write the dataframe to a SQLite table
df.to_sql('raw_data', conn, if_exists='replace', index=False)

25

In [5]:
# Create a function to aggregate data
def aggregate_data(group):
    result = {}
    for column in group.columns:
        if column != 'url':  # Skip the 'url' column
            unique_values = group[column].dropna().unique()
            if len(unique_values) == 1:
                result[column] = unique_values[0]
            elif len(unique_values) > 1:
                result[column] = list(unique_values)
            else:
                result[column] = None
    return pd.Series(result)

In [6]:
# Perform the aggregation
query = '''
SELECT *
FROM raw_data
'''
df = pd.read_sql_query(query, conn)
aggregated_df = df.groupby('url', as_index=False).apply(aggregate_data)



In [7]:
# Function to serialize data for SQLite
def serialize_data(value):
    if isinstance(value, (list, dict)):
        return json.dumps([serialize_data(v) for v in value] if isinstance(value, list) else value)
    elif isinstance(value, (np.integer, np.floating)):
        return int(value) if isinstance(value, np.integer) else float(value)
    elif isinstance(value, np.ndarray):
        return serialize_data(value.tolist())
    elif pd.isna(value):
        return None
    return value

In [8]:
# Apply serialization to all elements
aggregated_df = aggregated_df.applymap(serialize_data)

# Write the aggregated data to a new table
aggregated_df.to_sql('aggregated_data', conn, if_exists='replace', index=False)


4

In [9]:
print(aggregated_df)

                                                 url  \
0  https://www.energy-storage.news/goldman-sachs-...   
1  https://www.energy-storage.news/progress-repor...   
2  https://www.nsenergybusiness.com/news/canadian...   
3  https://www.nsenergybusiness.com/news/recurren...   

                            Unnamed: 0                                 owner  \
0                      [0, 1, 2, 3, 4]         Goldman Sachs Renewable Power   
1                 [20, 21, 22, 23, 24]  Goldman Sachs Renewable Power (GSRP)   
2                 [15, 16, 17, 18, 19]                      Recurrent Energy   
3  [5, 6, 7, 8, 9, 10, 11, 12, 13, 14]  Goldman Sachs Renewable Power (GSRP)   

                                            offtaker  storage_energy  \
0  ['Silicon Valley Clean Energy', 'Monterey Bay ...           561.0   
1  ['Silicon Valley Clean Energy', 'Monterey Bay ...           561.0   
2  ['Silicon Valley Clean Energy', 'Monterey Bay ...           180.0   
3  ['five different off-takers

In [13]:
aggregated_df = aggregated_df.drop('Unnamed: 0', axis=1)
aggregated_df.to_csv("aggregated_df_20240716.csv")